以下のサイトを利用した

https://dreamer-uma.com/mcmc-ising/

In [1]:
import os
import numpy as np
import random
import scipy as sp
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

関数の定義

In [2]:
class LatticeIsing_2D():
    def __init__(self, N, J, h):
        """
        Attributes:
            N: 格子サイズ
            J: 相互作用
            h: 磁場
        """
        self.N=N
        self.J=J
        self.h=h
    
    def neighor_spin_sum(self, state, x, y):

        center_spin = state[y, x]
        right, left, up, down=x+1, x-1, y+1, y-1

        # 周期境界条件
        if right>=self.N:
            right_spin=state[y, 0]
        else:
            right_spin=state[y, right]
            
        if left<0:
            left=self.N-1
            left_spin=state[y, left]
        else:
            left_spin=state[y, left]
            
        if down<0:
            down=self.N-1
            down_spin=state[down, x]
        else:
            down_spin=state[down, x]
    
        if up>=self.N:
            up_spin=state[0, x]
        else:
            up_spin=state[up, x]
            
        neighor_spin_sum=self.J * center_spin * (right_spin+left_spin+up_spin+down_spin)
        return neighor_spin_sum

    # エネルギーを計算
    def energy(self, state):
        energy=0
        for x in range(self.N):
            for y in range(self.N):
                energy -= self.neighor_spin_sum(state, x, y)/2
        energy -= self.h*np.sum(state)
        return energy
    
    # 磁化を計算
    def magnetization(self, state):
        return np.mean(state)

マルコフ連鎖モンテカルロ法（メトロポリス法，ギブスサンプリング）を定義

・gibbsサンプリングのコードが間違っているかも

In [3]:
class IsingMC():
    def __init__(self, method):
        self.method=method

    def run_mcmc(self, dist, init_state, beta, n_step):
        
        state=init_state
        state_dynamics=np.zeros((n_step+1, dist.N, dist.N))
        state_dynamics[0]=state
        AR_dynamics=[]

        for step in range(n_step):
            if self.method=="gibbs":
                state, AR = self.step_Gibbs(dist, state, beta)
            else:
                state, AR = self.step_MH(dist, state, beta)
            
            AR_dynamics.append(AR)
            state_dynamics[step+1] = state
        # print(f'ACCEPTANCE RATE : {np.mean(AR_dynamics):.3f}')
        return state_dynamics, AR_dynamics    
    
    # メトロポリス法
    def step_MH(self, dist, state, beta):
        n_flg = 0
        for i in range(dist.N * dist.N):
            x = np.random.randint(dist.N)
            y = np.random.randint(dist.N)
            delta_energy = 2 * (dist.neighor_spin_sum(state, x, y) + dist.h * state[y, x])
            trans_prob = np.exp(-beta * delta_energy)
            if delta_energy < 0:
                state[y, x] *= -1
                n_flg += 1
            elif np.random.random() <= trans_prob:
                state[y, x] *= -1
                n_flg += 1
        return state, n_flg / (dist.N * dist.N) 

    # ギブスサンプリング
    def step_Gibbs(self, dist, state, beta):
        # select rule
        flip_order_x = list(range(dist.N))
        flip_order_y = list(range(dist.N))
        random.shuffle(flip_order_x)
        random.shuffle(flip_order_y)
        for x in flip_order_x:
            for y in flip_order_y:
                neighors = (dist.neighor_spin_sum(state, x, y)+dist.h*state[y, x])/state[y, x]
                trans_prob = np.exp(beta*neighors)/(np.exp(beta*neighors)+np.exp(-beta*neighors))
                if np.random.random()<=trans_prob:
                    state[y, x] *=1
                else:
                    state[y, x] *=-1
        return state, 1.0

配位生成

In [5]:
spin_data = []  
label_data = []
# Ising Instance
N = 16
h = 0.0
J = 1
dist=LatticeIsing_2D(N, J, h)
# MCMC Instance
n_step_first = 1000     # 1つ目の配位生成
n_step_second = 500     # 2つ目以降の配位生成
sampler=IsingMC(method="metropolis")

num_label = 100     # ラベルの数
number = 100      # 1つの逆温度ラベルに対するスピンデータの数

# 0.2から1.0までを100等分した値で回す 
for i in tqdm(range(num_label)):  
    beta = 0.2 + (i / (num_label-1)) * 0.8   
    init_state = np.full((dist.N, dist.N), -1) # 初期配位
    state_dynamics, _ = sampler.run_mcmc(dist, init_state, beta, n_step=n_step_first)
    Ising_state = state_dynamics[n_step_first]
    
    # 各逆温度で1000個ずつ配位を作成   
    for j in range(number):
        init_state = Ising_state
        state_synamics, _ = sampler.run_mcmc(dist, init_state, beta, n_step=n_step_second)
        Ising_state = state_synamics[n_step_second]
        magnetization = Ising_state.sum()
        if magnetization > 0:
            Ising_state *= -1
        spin_data.append(Ising_state)
        label_data.append(beta)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [1:43:59<00:00, 62.40s/it]


データ読み込み

In [9]:
# 保存するデータをtupleにまとめる
data_to_save = (spin_data, label_data)

# フォルダ名とファイル名を指定
folder_name = "..\data_storage"
file_name = 'L16_beta.pkl'

# データをバイナリ形式で保存
file_path = os.path.join(folder_name, file_name)
with open(file_path, 'wb') as file:
    pickle.dump(data_to_save, file)